<a href="https://colab.research.google.com/github/Tynk3r/CM3005-Data-Science-Coursework-1/blob/main/CM3005_Data_Science_Midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Domain-specific area and objectives of the project
The area explored in this project is that of the realm of e-commmerce, specifically regarding the sale of laptops. Understanding the factors driving prices on laptops sold on e-commerce sites will benefit both consumers and sellers, and applying a linear regression model here will help us to understand the relationship between the prices of laptops and their specifications.

The objectives of this project are as follows:


1. Choose an appropriate dataset for this project (showing both the prices and specifications of laptops on e-commerce sites).
2. Clean the dataset for missing or unclean data and preprocess it to make it easier to visualise and apply a model to.
3. Construct a predictive model that will estimate laptop prices based on the specifications of the laptop.

This accurate pricing model will help consumers by allowing them to better assess the value of a given laptop based on its specifications and this facilitate more informed purchases. It will also help sellers by letting them understand the factors affecting laptop pricing in the market, allowing them to make more strategic pricing strategies.

The significance of this project in this domain-specific area is that a more accurate model of the prices of laptops will lead to more transparent and competitive markets. It also contributes to the ongoing conversation around leveraging machine learning in e-commerce.

# 2. Dataset description

The dataset chosen for this project is a collection of laptop product listings from an e-commerce website, with its specifications and price listed. There are 1300 listings provided in the dataset, and the data types provided are as follows:
* Categorical Variables: Brand, Type, CPU, GPU, OS
* Numerical Variables: Screen Size, Resolution, RAM, Storage Capacity, Weight
* Target Variable: Price (continuous)

The source of the dataset is from Kaggle.com: https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset
